In [2]:
from __future__ import annotations
from pathlib import Path

from scraping_kit import load_db_and_bots

from scraping_kit.utils import date_one_day

path_data = Path("data")
db_tw, bots = load_db_and_bots(path_data, "scrape_tw")

Collection Names: ['raw', 'trends']
Bots: bots=[BotScraper(acc_name='Vera')]


In [ ]:
db_tw.db.drop_collection("raw")
db_tw.db.drop_collection("trends")

In [ ]:
date = date_one_day(2023, 12, 28)
date

In [ ]:
#trends = db_tw.coll.trends_from_insert_id(insert_one_result_trend)     # --> Search by id in the DB.
trends = db_tw.coll.trends_from_date_range(date)                        # --> Search by date range, returns the first result found in that range.

df_trends = trends.get_df()
df_trends